In [2]:
import pickle
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import contextlib
from selenium.webdriver.support.ui import Select


import chromedriver_binary

In [3]:
with open('pickle/midland_apis.pickle', 'rb') as to_read: #run scrape_data to generate
    midland_apis = pickle.load(to_read)

In [4]:
midland_apis.head()

0    42329422450000
1    42227347100000
2    42227347130000
3    42329415170000
4    42317408540000
Name: APINumber, dtype: int64

In [5]:
@contextlib.contextmanager
def quitting(thing):
    yield thing
    #thing.close()
    #thing.quit()

In [6]:
def proration_query(api):
    proration_url = 'http://webapps2.rrc.texas.gov/EWA/oilProQueryAction.do'
    api_dict = {'api' : api}
    api = str(api)
    api_mid = str(api)[2:5]
    api_last = str(api)[5:10]
    
    with quitting(webdriver.Chrome()) as driver:
        driver.get(proration_url)
        api_box1 = driver.find_element_by_name("searchArgs.apiPrefixArg")
        api_box1.send_keys(api_mid)
        api_box2 = driver.find_element_by_name("searchArgs.apiSuffixArg")
        api_box2.send_keys(api_last)
        pro_search = driver.find_element_by_xpath("/html/body/table[2]/tbody/tr/td/form/table/tbody/tr[2]/td/table/tbody/tr[3]/td/input[1]")
        pro_search.click()
        status = driver.find_element_by_xpath("/html/body/table[2]/tbody/tr/td/form/table/tbody/tr[2]/td/table/tbody/tr[2]/td/table/tbody/tr[4]/td[16]").text
        api_dict['status'] = status
        if status == 'SHUT IN':
            return api_dict
        w10_link = driver.find_element_by_xpath("/html/body/table[2]/tbody/tr/td/form/table/tbody/tr[2]/td/table/tbody/tr[2]/td/table/tbody/tr[4]/td[5]/a")
        w10_link.click()

        for tr in driver.find_elements_by_xpath('/html/body/table[2]/tbody/tr/td/form/table/tbody/tr[5]/td/table/tbody/tr[1]/td/table[2]//tr'):
            tds = tr.find_elements_by_tag_name('td')
            if tds: 
                api_dict[tds[0].text] = tds[1].text
        ret_but = driver.find_element_by_name("unused")
        ret_but.click()
        select_element = Select(driver.find_element_by_name("propertyValue"))
        select_element.select_by_visible_text("Completion")
        completion_reports = driver.find_elements_by_xpath("/html/body/table[5]/tbody/tr/td/form/table/tbody/tr[3]/td/table/tbody/tr[2]/td/table/tbody//tr")
        last_completion = completion_reports[-1].find_elements_by_tag_name('td')[0]
        last_completion.click()
        
        completion_type = driver.find_element_by_xpath("/html/body/table[5]/tbody/tr/td/form/table/tbody/tr/td/table[2]/tbody/tr[2]/td[1]/table/tbody/tr[1]/td/table/tbody/tr/td[1]/table/tbody/tr[3]/td[2]/strong").text
        api_dict['completion type'] = completion_type
        
        direction = driver.find_element_by_xpath("/html/body/table[5]/tbody/tr/td/form/table/tbody/tr/td/table[2]/tbody/tr[2]/td[1]/table/tbody/tr[1]/td/table/tbody/tr/td[1]/table/tbody/tr[8]/td[2]/strong").text
        api_dict['direction'] = direction
        
        comments = driver.find_element_by_name("publicComments").text
        api_dict['comments'] = comments
    return api_dict

In [7]:
proration_query(midland_apis[400])

{'api': 42227394730000,
 'status': 'PRODUCING',
 'Test Type:': 'Survey',
 'Test Date:': '10/12/2018',
 'Effective Date:': '04/01/2019',
 'Type of Producing Method:': 'Pumping',
 'Daily Oil BBL:': '631.0',
 'Daily Water BBL:': '3788',
 'Daily Gas MCF:': '352',
 'Gas/Oil Ratio CF/BBL:': '557',
 'Allocation:': 'No',
 'completion type': 'New Well',
 'direction': 'HORIZONTAL',
 'comments': '[RRC Staff 2018-12-14 16:40:59.932]: SWR 37 LL Exception. location of well is plotted to be OK with take points at 7772-17389 feet; [RRC Staff 2018-12-14 16:41:16.176]: EDL=9600 feet, max acres=640, SPRABERRY (TREND AREA) oil well; \n\ntake points: 7772-17389 feet; '}